In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X=torch.randn(2,20)
net(X)

tensor([[-0.0241, -0.3848, -0.6349,  0.0947,  0.0906, -0.1199, -0.3115,  0.3495,
         -0.1794,  0.1520],
        [-0.0744,  0.2651, -0.4299, -0.0009,  0.0577, -0.0889,  0.0272,  0.0768,
          0.1308, -0.0941]], grad_fn=<AddmmBackward0>)

## 5.1.1自定义块

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层这里，声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden=nn.Linear(20,256)   # 隐藏层
        self.out=nn.Linear(256,10)    # 输出层
    # 定义函数的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        # 注意，这里使用ReLu的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))
    
net=MLP()
net(X)

tensor([[-9.0282e-02,  8.3717e-01,  9.7856e-02,  1.2094e-01,  1.0872e-01,
         -1.7735e-01,  4.6022e-02,  1.6405e-01, -3.8527e-01, -2.8815e-01],
        [ 9.1108e-03,  5.7216e-01,  1.0787e-02,  4.5813e-04,  3.1982e-01,
         -5.2695e-02, -6.8458e-03, -9.8541e-03, -4.5075e-01, -6.7615e-01]],
       grad_fn=<AddmmBackward0>)

## 5.1.2顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # 这里，module是Module子类的一个实例我们把它保存在Module类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)]=module
    def forward(self,X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X=block(X)
        return X
    
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0138, -0.2686,  0.1673, -0.3724, -0.0462, -0.3816,  0.1561,  0.1268,
          0.3489,  0.1255],
        [-0.0246, -0.6716, -0.1847,  0.0671, -0.0239, -0.1628, -0.2983,  0.3357,
          0.1598, -0.1904]], grad_fn=<AddmmBackward0>)

## 5.1.3在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此其在训练期间保持不变
        self.rand_weight=torch.randn((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # 复用全连接层这相当于两个全连接层共享参数
        X=self.linear(X)
        # 控制流
        while X.abs().sum() >1:
            X/=2
        return X.sum()
    
net=FixedHiddenMLP()
net(X)

tensor(0.1226, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
    
chimera=nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.0967, grad_fn=<SumBackward0>)